# Attribution Data Processing Pipeline

This notebook executes the complete data preparation workflow for analyzing model attribution scores. It transforms raw experimental logs (JSON and `.dill` files) into a structured, aggregated, and filtered dataset suitable for comparative analysis.

The pipeline processes the data in five sequential stages:

1.  **Initialization**
    Loads the raw experimental predictions from `solutions.json` and converts them into a long-format Pandas DataFrame, establishing the base structure for analysis.

2.  **Chronological Sorting**
    Organizes the text spans within each task chronologically (e.g., ensuring "User Prompt" appears before "Action Step") while preserving the original file order.

3.  **Attribution Aggregation**
    Iterates through the raw Inseq output files (`_inseq_out.dill`) to compute aggregated attribution scores (Mean, Max, Sum, AbsMax, etc.) for every defined text span (Plan and Action).

4.  **Decoy Merging**
    Consolidates split spans (specifically `decoy_functions_1` and `decoy_functions_2`) into single logical units, mathematically combining their attribution scores to create a unified "Decoy Tools" metric.

5.  **Filtering & Normalization**
    Performs final cleanup by filtering for tasks where data exists for *both* models (ensuring fair comparison) and reordering columns for readability and logical flow.



---
**Input Data:** `data/solutions.json` and raw files in `outputs/`
**Final Output:** `data/solutions_dataframe_normalized.parquet`

Create Dataframe to save solutions

In [16]:
import os
import json
import pandas as pd
import numpy as np

def initialize_and_save_dataframe(json_filepath: str, output_filepath: str):
    """
    Loads experimental data from a JSON file, structures it into a long-format
    pandas DataFrame, and saves it to a Parquet file.

    This version safely handles predictions that may be missing a 'spans' dictionary.
    """
    
    # Load the entire JSON file into memory
    with open(json_filepath, 'r') as f:
        solutions_data = json.load(f)

    all_rows = []

    # --- Iterate through the nested JSON structure ---
    for record in solutions_data:
        task_id = str(record['id'])
        category = record['category']

        for prediction in record['predictions']:
            model = prediction['model']
            filename = prediction['file']

            # 3. Inner loop: Iterate safely through spans, handling missing keys
            for span_name, span_coords in prediction.get('spans', {}).items():
                span_start, span_end = span_coords
                span_len = span_end - span_start

                # Create a dictionary representing a single row in our DataFrame
                row = {
                    "task_id": task_id,
                    "model": model,
                    "category": category,
                    "filename": filename,
                    "span_name": span_name,
                    "span_start": span_start,
                    "span_end": span_end,
                    "span_len": span_len,
                    "attr_plan": np.nan,
                    "attr_action": np.nan
                }
                all_rows.append(row)

    # Create the DataFrame from the list of rows
    df = pd.DataFrame(all_rows)

    # Reorder columns to match the desired final layout
    column_order = [
        "task_id", "model", "category", "filename", 
        "span_name", "span_start", "span_end", "span_len",
        "attr_plan", "attr_action"
    ]
    df = df[column_order]

    # Ensure the output directory exists
    output_dir = os.path.dirname(output_filepath)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # Save the DataFrame to a Parquet file for efficiency and type safety
    df.to_parquet(output_filepath, index=False)
    print(f"DataFrame successfully created and saved to {output_filepath}")

if __name__ == "__main__":
    json_filepath = "../data/solutions.json"
    output_filepath = "../data/solutions_dataframe_new.parquet"

    initialize_and_save_dataframe(json_filepath, output_filepath)

Sort spans in dataframe object

In [23]:
import pandas as pd
import os

def sort_spans_and_save(input_filepath: str, output_filepath: str):
    """
    Loads a DataFrame, sorts the spans within each filename group by their
    start position, and saves the result to a new file.

    The original order of the filename groups is preserved.

    Args:
        input_filepath (str): The path to the source Parquet file.
        output_filepath (str): The path to save the sorted Parquet file.
    """
    if not os.path.exists(input_filepath):
        print(f"Error: Input file not found at {input_filepath}")
        return None

    print(f"Loading data from {input_filepath}...")
    dataframe = pd.read_parquet(input_filepath)

    original_filename_order = dataframe['filename'].unique()
    dataframe['filename'] = pd.Categorical(
        dataframe['filename'],
        categories=original_filename_order,
        ordered=True
    )

    # Sort by filename first (preserving original block order) and
    # then by span_start to order the rows within each block.
    print("Sorting spans within each filename group...")
    sorted_dataframe = dataframe.sort_values(by=['filename', 'span_start'])

    # Convert the filename column back to a regular string object for saving
    sorted_dataframe['filename'] = sorted_dataframe['filename'].astype(str)

    # Ensure the output directory exists
    output_dir = os.path.dirname(output_filepath)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # Save the newly sorted DataFrame
    sorted_dataframe.to_parquet(output_filepath, index=False)
    print(f"Sorted DataFrame successfully saved to {output_filepath}")

    return sorted_dataframe

if __name__ == "__main__":
    # Define your input and output paths here
    input_path = "../data/solutions_dataframe_new.parquet"
    output_path_sorted = "../data/solutions_dataframe_new.parquet"

    # Run the sorting function
    sort_spans_and_save(input_path, output_path_sorted)

Unsorted, before:      task_id       model category  \
5033    2100  Qwen3-0.6B     data   
5034    2100  Qwen3-0.6B     data   
5035    2100  Qwen3-0.6B     data   
5036    2100  Qwen3-0.6B     data   
5037    2100  Qwen3-0.6B     data   
5038    2100  Qwen3-0.6B     data   
5039    2100  Qwen3-0.6B     data   
5040    2100  Qwen3-0.6B     data   
5041    2100  Qwen3-0.6B     data   
5042    2100  Qwen3-0.6B     data   
5043    2100  Qwen3-0.6B     data   
5044    2100  Qwen3-0.6B     data   

                                               filename  is_correct  \
5033  data_ID2100_Qwen3-0.6B_21-09_10-24-45_agent_ou...       False   
5034  data_ID2100_Qwen3-0.6B_21-09_10-24-45_agent_ou...       False   
5035  data_ID2100_Qwen3-0.6B_21-09_10-24-45_agent_ou...       False   
5036  data_ID2100_Qwen3-0.6B_21-09_10-24-45_agent_ou...       False   
5037  data_ID2100_Qwen3-0.6B_21-09_10-24-45_agent_ou...       False   
5038  data_ID2100_Qwen3-0.6B_21-09_10-24-45_agent_ou...       False   
503

Aggregate Inseq Spans and save to dataframe 

In [33]:
import os
import dill
import pandas as pd
from tqdm.auto import tqdm 


from inseq.data.aggregator import AggregatorPipeline, ContiguousSpanAggregator

def aggregate_and_populate(
    sorted_dataframe: pd.DataFrame,
    base_path: str = "../outputs/"
) -> pd.DataFrame:
    """
    Performs inseq span aggregation for every unique file in the DataFrame and
    populates the DataFrame with the attribution scores.

    Args:
        sorted_dataframe (pd.DataFrame): The pre-sorted DataFrame from the previous step.
        base_path (str): The root directory where inseq output folders are located.

    Returns:
        pd.DataFrame: The DataFrame, now populated with attribution scores.
    """
    # List of methods aggregation methods
    methods = ['absmax', 'prod', 'min', 'vnorm', 'mean', 'sum', 'max']

    # --- 1. Prepare the DataFrame by adding new columns ---
    print("Preparing DataFrame with new attribution columns...")
    for method in methods:
        sorted_dataframe[f"{method}_plan_attr"] = pd.NA
        sorted_dataframe[f"{method}_action_attr"] = pd.NA

    # Get a list of unique filenames to iterate over
    unique_files = sorted_dataframe['filename'].unique()
    print(f"Found {len(unique_files)} unique files to process.")

    # --- 2. Main loop: Iterate through each unique file ---
    for filename in tqdm(unique_files, desc="Aggregating Files"):
        # Get all rows corresponding to the current file
        file_specific_rows = sorted_dataframe[sorted_dataframe['filename'] == filename]
        
        # --- 3. Construct the path and load the inseq attribution file ---
        category = file_specific_rows['category'].iloc[0]
        inseq_filename = filename.replace("_agent_out.dill", "_inseq_out.dill")
        inseq_filepath = os.path.join(base_path, category, inseq_filename)

        try:
            with open(inseq_filepath, 'rb') as f:
                loaded_attributions = dill.load(f)
        except (FileNotFoundError, dill.UnpicklingError) as e:
            print(f"\nWarning: Could not load or parse {inseq_filepath}. Skipping. Error: {e}")
            continue

        # --- 4. Prepare spans and names for the aggregator ---
        target_spans = list(zip(
            file_specific_rows['span_start'],
            file_specific_rows['span_end']
        ))
        target_names = file_specific_rows['span_name'].tolist()

        # --- 5. Inner loop: Run aggregation for each method ---
        for method in methods:
            # Initialize the pipeline with the current method
            span_pipeline = AggregatorPipeline(
               [ContiguousSpanAggregator],
               [method]
            )

            # Aggregate the attributions
            token_aggregated = loaded_attributions.sequence_attributions[0].aggregate(
                aggregator=span_pipeline,
                target_spans=target_spans
            )
            
            # Name the aggregated spans
            for i, name in enumerate(target_names):
                token_aggregated.target[i].token = name

            # --- 6. Extract results and save to the main DataFrame ---
            # The result is a tensor with shape [num_spans, 2]
            attr_tensor = token_aggregated.target_attributions

            # Get the indices in the main DataFrame where we need to save the data
            target_indices = file_specific_rows.index

            # Define the column names for the current method
            plan_col = f"{method}_plan_attr"
            action_col = f"{method}_action_attr"

            # Update the DataFrame using.loc for safe and efficient assignment
            sorted_dataframe.loc[target_indices, plan_col] = attr_tensor[:, 0].tolist()
            sorted_dataframe.loc[target_indices, action_col] = attr_tensor[:, 1].tolist()

    print("\nAggregation and data population complete.")
    return sorted_dataframe

if __name__ == "__main__":
    # 1. Load the sorted DataFrame from the previous step
    input_path = "../data/solutions_dataframe_new.parquet"
    print(f"Loading sorted DataFrame from {input_path}...")
    df_sorted = pd.read_parquet(input_path)

    # 2. Run the main aggregation function
    df_final = aggregate_and_populate(df_sorted)

    # 3. Save the final, populated DataFrame
    output_path_final = "../data/solutions_dataframe_aggregated.parquet"
    print(f"Saving final populated DataFrame to {output_path_final}...")
    df_final.to_parquet(output_path_final, index=False)

    # 4. Verify the result
    print("\n--- Verification ---")
    print("Final DataFrame Info (note the new columns):")
    df_final.info()
    print("\nFirst 5 rows of the final DataFrame:")
    print(df_final.head())

Loading sorted DataFrame from data/solutions_dataframe_sorted.parquet...
Preparing DataFrame with new attribution columns...
Found 482 unique files to process.


Aggregating Files:   0%|          | 0/482 [00:00<?, ?it/s]


Aggregation and data population complete.
Saving final populated DataFrame to data/solutions_dataframe_final.parquet...

--- Verification ---
Final DataFrame Info (note the new columns):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5760 entries, 0 to 5759
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   task_id             5760 non-null   object 
 1   model               5760 non-null   object 
 2   category            5760 non-null   object 
 3   filename            5760 non-null   object 
 4   is_correct          5760 non-null   bool   
 5   span_name           5760 non-null   object 
 6   span_start          5760 non-null   int64  
 7   span_end            5760 non-null   int64  
 8   span_len            5760 non-null   int64  
 9   attr_plan           0 non-null      float64
 10  attr_action         0 non-null      float64
 11  absmax_plan_attr    5278 non-null   object 
 12  absmax_action_

Clean dataframe and combine "decoy_function"

In [7]:
import pandas as pd
import numpy as np
import math
from functools import reduce
from tqdm.auto import tqdm

# --- Part 1: Aggregation Logic Helper Function ---

def combine_values(val1: float, val2: float, len1: int, len2: int, method: str) -> float:
    """
    Combines two attribution values using a specified aggregation method.

    Args:
        val1 (float): The value from the first span.
        val2 (float): The value from the second span.
        len1 (int): The length of the first span.
        len2 (int): The length of the second span.
        method (str): The aggregation method name.

    Returns:
        float: The combined value.
    """
    if method == 'sum':
        return val1 + val2
    elif method == 'mean':
        total_length = len1 + len2
        if total_length == 0: return 0
        weighted_sum = (val1 * len1) + (val2 * len2)
        return weighted_sum / total_length
    elif method == 'max':
        return max(val1, val2)
    elif method == 'min':
        return min(val1, val2)
    elif method == 'prod':
        return val1 * val2
    elif method == 'absmax':
        return val1 if abs(val1) >= abs(val2) else val2
    elif method == 'vnorm':  # Assumes L2 norm
        return math.sqrt(val1**2 + val2**2)
    else:
        # Fallback for any unexpected method, though this shouldn't be hit
        return np.nan

# --- Part 2: Main Script to Load, Clean, and Aggregate ---

# 1. Load the DataFrame
final_filepath = "../data/solutions_dataframe_aggregated.parquet"
print(f"Loading DataFrame from {final_filepath}...")
df = pd.read_parquet(final_filepath)
print("DataFrame loaded successfully.")

# 2. Delete initial placeholder columns if they are empty
cols_to_delete = ["attr_plan", "attr_action"]
for col in cols_to_delete:
    if col in df.columns and df[col].isnull().all():
        print(f"Column '{col}' exists and is empty. Deleting it.")
        df = df.drop(columns=[col])
    else:
        print(f"Column '{col}' does not exist or is not empty. No action taken.")

# 3. Prepare for aggregation
print("\nStarting aggregation of 'decoy_functions' spans...")
aggregation_methods = ['absmax', 'prod', 'min', 'vnorm', 'mean', 'sum', 'max']
processed_rows = []

# Group by 'filename' to process each file's spans independently
grouped = df.groupby('filename')

for filename, group_df in tqdm(grouped, desc="Processing files"):
    # Isolate the decoy spans and all other spans
    decoy1 = group_df[group_df['span_name'] == 'decoy_functions_1']
    decoy2 = group_df[group_df['span_name'] == 'decoy_functions_2']
    other_rows = group_df[~group_df['span_name'].isin(['decoy_functions_1', 'decoy_functions_2'])]

    # This list will hold the new/modified rows for this group
    current_group_new_rows = [other_rows]

    # Case 1: Both decoy functions exist, so we must combine them
    if not decoy1.empty and not decoy2.empty:
        d1_row = decoy1.iloc[0]
        d2_row = decoy2.iloc[0]

        # Start building the new combined row
        new_row = {
            "task_id": d1_row['task_id'],
            "model": d1_row['model'],
            "category": d1_row['category'],
            "filename": d1_row['filename'],
            "span_name": "decoy_functions",
            "span_start": np.nan,
            "span_end": np.nan,
            "span_len": d1_row['span_len'] + d2_row['span_len']
        }

        # Apply the combination algorithm for each attribution method
        for method in aggregation_methods:
            # Combine the 'plan' attributes
            new_row[f'{method}_plan_attr'] = combine_values(
                d1_row[f'{method}_plan_attr'], d2_row[f'{method}_plan_attr'],
                d1_row['span_len'], d2_row['span_len'], method
            )
            # Combine the 'action' attributes
            new_row[f'{method}_action_attr'] = combine_values(
                d1_row[f'{method}_action_attr'], d2_row[f'{method}_action_attr'],
                d1_row['span_len'], d2_row['span_len'], method
            )
        
        current_group_new_rows.append(pd.DataFrame([new_row]))

    # Case 2: Only one of the decoy functions exists, so we just rename it
    elif not decoy1.empty:
        renamed_decoy = decoy1.copy()
        renamed_decoy['span_name'] = 'decoy_functions'
        current_group_new_rows.append(renamed_decoy)
    elif not decoy2.empty:
        renamed_decoy = decoy2.copy()
        renamed_decoy['span_name'] = 'decoy_functions'
        current_group_new_rows.append(renamed_decoy)

    # Add the processed rows for this group to our main list
    processed_rows.append(pd.concat(current_group_new_rows))

# 4. Rebuild the final DataFrame
print("\nRebuilding final DataFrame from processed rows...")
final_aggregated_df = pd.concat(processed_rows, ignore_index=True)

# 5. Save the new DataFrame
output_path = "../data/solutions_dataframe_aggregated.parquet"
final_aggregated_df.to_parquet(output_path, index=False)
print(f"Aggregation complete. New DataFrame saved to {output_path}")

# --- Verification ---
print("\n--- Verification ---")
print("Shape of new DataFrame:", final_aggregated_df.shape)
print("Span names in the new DataFrame:", final_aggregated_df['span_name'].unique())

# Check a sample file to see the result
sample_filename = final_aggregated_df['filename'].iloc[0]
print(f"\nChecking spans for sample file: {sample_filename}")
print(final_aggregated_df[final_aggregated_df['filename'] == sample_filename][['span_name', 'span_len', 'sum_plan_attr']])

Loading DataFrame from data/solutions_dataframe_final.parquet...
DataFrame loaded successfully.
Column 'attr_plan' exists and is empty. Deleting it.
Column 'attr_action' exists and is empty. Deleting it.

Starting aggregation of 'decoy_functions' spans...


Processing files:   0%|          | 0/482 [00:00<?, ?it/s]


Rebuilding final DataFrame from processed rows...
Aggregation complete. New DataFrame saved to data/solutions_dataframe_aggregated.parquet

--- Verification ---
Shape of new DataFrame: (5302, 23)
Span names in the new DataFrame: ['agentic_behaviour_instructions' 'few_shot_examples' 'functions_intro'
 'correct_function' 'final_answer_tool' 'general_code_use_rules'
 'call_to_action' 'user_prompt' 'planning_step' 'action_step'
 'decoy_functions']

Checking spans for sample file: business_and_productivity_ID1001_Qwen3-0.6B_17-09_16-19-14_agent_out.dill
                         span_name  span_len  sum_plan_attr
0   agentic_behaviour_instructions       211      29.680502
1                few_shot_examples      1254      47.577736
2                  functions_intro        42       2.117552
3                 correct_function        75       8.512258
4                final_answer_tool        39       2.686046
5           general_code_use_rules       347      30.701908
6                   call

Remove Task IDs that dont have both models. Reorder methods.

In [1]:
import pandas as pd
import os

def print_dataframe_info(df: pd.DataFrame, title: str):
    """
    Prints a detailed summary of a DataFrame's properties.
    """
    print(f"\n--- {title} ---")
    
    # Basic info
    num_rows, num_cols = df.shape
    print(f"Number of rows: {num_rows}")
    print(f"Number of columns: {num_cols}")
    print("Column names:", df.columns.tolist())
    
    # Uniqueness stats
    unique_tasks = df['task_id'].nunique()
    print(f"\nNumber of unique 'task_id' entries: {unique_tasks}")
    
    # Calculated averages
    if not df.empty:
        avg_models_per_task = df.groupby('task_id')['model'].nunique().mean()
        avg_spans_per_file = df.groupby('filename')['span_name'].nunique().mean()
        print(f"Average number of unique 'model' per 'task_id': {avg_models_per_task:.2f}")
        print(f"Average number of 'span_name' per 'filename': {avg_spans_per_file:.2f}")
    
    print("-" * (len(title) + 8))

# --- Main Script ---

# 1. Load the aggregated DataFrame
aggregated_filepath = "data/solutions_dataframe_aggregated.parquet"
print(f"Loading aggregated DataFrame from {aggregated_filepath}...")
df_agg = pd.read_parquet(aggregated_filepath)

# 2. Print info for the original, unfiltered DataFrame
print_dataframe_info(df_agg, "Original DataFrame Info (Before Filtering)")

# 3. Filter for tasks that have entries for two different models
model_counts_per_task = df_agg.groupby('task_id')['model'].nunique()
tasks_with_both_models = model_counts_per_task[model_counts_per_task == 2].index
df_filtered = df_agg[df_agg['task_id'].isin(tasks_with_both_models)].copy()

# 4. Display the number of task IDs that have both models
num_tasks_with_both = len(tasks_with_both_models)
print(f"\n✅ Found {num_tasks_with_both} task IDs with data for both models.")

# 5. Print info for the newly filtered DataFrame
print_dataframe_info(df_filtered, "Filtered DataFrame Info (Tasks with Both Models Only)")

# 6. Reorder the columns
print("\n--- Reordering Columns ---")
print("\nOriginal column order:")
print(df_filtered.columns.tolist())

# Define the desired order for the attribution columns
ordered_attr_cols = [
    'vnorm_plan_attr', 'vnorm_action_attr', 'mean_plan_attr', 'mean_action_attr',
    'prod_plan_attr', 'prod_action_attr', 'absmax_plan_attr', 'absmax_action_attr',
    'max_plan_attr', 'max_action_attr', 'min_plan_attr', 'min_action_attr',
    'sum_plan_attr', 'sum_action_attr'
]

# Get the list of all other columns, preserving their original order
front_cols = [col for col in df_filtered.columns if col not in ordered_attr_cols]

# Combine the two lists to create the final column order
new_column_order = front_cols + ordered_attr_cols

# Apply the new order to the DataFrame
df_normalized = df_filtered[new_column_order]

print("\nNew column order:")
print(df_normalized.columns.tolist())
print("--------------------------")

# 7. Save the new, normalized DataFrame
output_dir = "../data/"
output_filename = "solutions_dataframe_normalized.parquet"
output_filepath = os.path.join(output_dir, output_filename)

os.makedirs(output_dir, exist_ok=True)
df_normalized.to_parquet(output_filepath, index=False)

print(f"\n✅ Filtered and reordered DataFrame saved successfully to: {output_filepath}")

Loading aggregated DataFrame from data/solutions_dataframe_aggregated.parquet...

--- Original DataFrame Info (Before Filtering) ---
Number of rows: 5302
Number of columns: 23
Column names: ['task_id', 'model', 'category', 'filename', 'is_correct', 'span_name', 'span_start', 'span_end', 'span_len', 'absmax_plan_attr', 'absmax_action_attr', 'prod_plan_attr', 'prod_action_attr', 'min_plan_attr', 'min_action_attr', 'vnorm_plan_attr', 'vnorm_action_attr', 'mean_plan_attr', 'mean_action_attr', 'sum_plan_attr', 'sum_action_attr', 'max_plan_attr', 'max_action_attr']

Number of unique 'task_id' entries: 250
Average number of unique 'model' per 'task_id': 1.93
Average number of 'span_name' per 'filename': 11.00
--------------------------------------------------

✅ Found 232 task IDs with data for both models.

--- Filtered DataFrame Info (Tasks with Both Models Only) ---
Number of rows: 5104
Number of columns: 23
Column names: ['task_id', 'model', 'category', 'filename', 'is_correct', 'span_nam